In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import BinaryAccuracy
def criar_cnn(input_shape):
    # 1. Definir a arquitetura da CNN
    model = models.Sequential()

    # Primeira camada convolutiva
    model.add(layers.Conv1D(filters=4, kernel_size=1, activation='relu', input_shape=input_shape))
    model.add(layers.Conv1D(filters=8, kernel_size=1, activation='relu', input_shape=input_shape))
    model.add(layers.Conv1D(filters=16, kernel_size=1, activation='relu', input_shape=input_shape))


    # Segunda camada convolutiva
    model.add(layers.Conv1D(filters=8, kernel_size=1, activation='relu'))


    # Flatten para conectar às camadas totalmente conectadas
    model.add(layers.Flatten())

    # Camada totalmente conectada
    model.add(layers.Dense(16, activation='relu'))

    # Camada de saída com função de ativação sigmoid (classificação binária)
    model.add(layers.Dense(1, activation='sigmoid'))

    # 2. Compilar o modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[BinaryAccuracy(threshold=0.5)])

    # 3. Resumo do modelo
    model.summary()
    
    return model


In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Carregar os dados do CSV
df = pd.read_csv('dados_de_treino.csv')

# Converter as labels de 'gol' para 0 e 1
df['gol'] = df['gol'].astype(int)

# Selecionar as features (distância e ângulo) e a label (gol)
X = df[['dist', 'ang']].values
y = df['gol'].values

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape para adaptar ao input da CNN (adicionando dimensão de canais)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Criar o modelo
input_shape = (X.shape[1], 1)
import numpy as np
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import brier_score_loss, log_loss

# Separar dados de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar a CNN com o conjunto de treinamento
model = criar_cnn(input_shape)
model.fit(X_train, y_train, epochs=100, batch_size=8)

# Obter as previsões de probabilidade (antes da calibration)
y_pred_probs = model.predict(X_val)

# Converter previsões para 1D
y_pred_probs = y_pred_probs.flatten()

# Platt Scaling (logistic regression para calibration)
platt_scaler = LogisticRegression()
platt_scaler.fit(y_pred_probs.reshape(-1, 1), y_val)

# Calibrar as previsões
y_pred_probs_platt = platt_scaler.predict_proba(y_pred_probs.reshape(-1, 1))[:, 1]

# Avaliar a performance das previsões calibradas (opcional)
print("Log Loss após Platt Scaling:", log_loss(y_val, y_pred_probs_platt))
print("Brier Score após Platt Scaling:", brier_score_loss(y_val, y_pred_probs_platt))

# Alternativamente, aplicar isotonic regression para calibration
iso_scaler = IsotonicRegression(out_of_bounds='clip')
iso_scaler.fit(y_pred_probs, y_val)

# Calibrar as previsões com isotonic regression
y_pred_probs_iso = iso_scaler.transform(y_pred_probs)

# Avaliar a performance das previsões calibradas com isotonic regression (opcional)
print("Log Loss após Isotonic Regression:", log_loss(y_val, y_pred_probs_iso))
print("Brier Score após Isotonic Regression:", brier_score_loss(y_val, y_pred_probs_iso))


# Salvar o modelo treinado
model.save('modelo_cnn_xG.h5')
print("Modelo salvo como 'modelo_cnn_xG.h5'.")
# Salvar o scaler para futura normalização de novos dados


c:\Users\pedro\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_26 (Conv1D)              │ (None, 2, 4)           │             8 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_27 (Conv1D)              │ (None, 2, 8)           │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_28 (Conv1D)              │ (None, 2, 16)          │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_29 (Conv1D)              │ (None, 2, 8)           │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 617 (2.41 KB)

 Trainable params: 617 (2.41 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - binary_accuracy: 0.8792 - loss: 0.6131
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8749 - loss: 0.3931
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8550 - loss: 0.4189
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8596 - loss: 0.3996
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8577 - loss: 0.3982
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8792 - loss: 0.3528
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8683 - loss: 0.3647
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8636 - loss: 0.3750
Epoch 9/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8784 - loss: 0.3552
Epoch 10/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8684 - loss: 0.3680
Epoch 11/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - bin

Log Loss após Platt Scaling: 0.3961014245387854
Brier Score após Platt Scaling: 0.11717540577878288
Log Loss após Isotonic Regression: 0.37280330575533915
Brier Score após Isotonic Regression: 0.11058781152167559
Modelo salvo como 'modelo_cnn_xG.h5'.


In [29]:
import tensorflow as tf
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression

# Carregar o modelo salvo
model = tf.keras.models.load_model('modelo_cnn_xG.h5')


# Carregar os dados de teste do CSV
df_teste = pd.read_csv('dados_de_teste.csv')

# Selecionar as features (distância e ângulo)
X_teste = df_teste[['dist', 'ang']].values

# Normalizar os dados usando o scaler carregado
X_teste = scaler.transform(X_teste)

# Reshape para adaptar ao input da CNN (adicionando dimensão de canais)
X_teste = X_teste.reshape((X_teste.shape[0], X_teste.shape[1], 1))

# Fazer as previsões com o modelo carregado
xG_pred = model.predict(X_teste)

# Se for aplicar calibração, ajuste as previsões
# Descomente a linha correspondente ao método de calibração que você está usando
# xG_pred_calibrated = platt_scaler.predict_proba(xG_pred)[:, 1]  # Para Platt Scaling
xG_pred_calibrated = iso_scaler.transform(xG_pred.flatten())  # Para Isotonic Regression

# Adicionar as previsões calibradas como uma nova coluna no DataFrame
df_teste['xG'] = xG_pred_calibrated

# Salvar o DataFrame atualizado em um novo arquivo CSV
df_teste.to_csv('dados_de_teste_com_xG.csv', index=False)

print("Previsões feitas e coluna 'xG' adicionada ao arquivo 'dados_de_teste_com_xG.csv'.")



15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Previsões feitas e coluna 'xG' adicionada ao arquivo 'dados_de_teste_com_xG.csv'.


In [30]:
import pandas as pd

# Carregar o arquivo CSV com as previsões
df = pd.read_csv('dados_de_teste_com_xG.csv')

# Selecionar apenas as colunas 'shot_id' e 'xG'
df_submission = df[['shot_id', 'xG']]

# Salvar o novo DataFrame em um arquivo CSV chamado 'submission.csv'
df_submission.to_csv('submission.csv', index=False)

print("Arquivo 'submission.csv' criado com sucesso.")


Arquivo 'submission.csv' criado com sucesso.
